In [1]:
from mdagent import MDAgent
import os 
import mdtraj as md

In [2]:
import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(parent_dir)
from robustness_prompts import get_prompt

prompt_6_descriptive = get_prompt("descriptive", 6)
prompt_6_natural = get_prompt("natural", 6)

prompt_6_descriptive, prompt_6_natural

('Complete all of the following tasks: 1. Download the PDB file 1LYZ. 2. Report the secondary structure assignments of the downloaded PDB structure. 3. Simulate for 1ps at 300 K. 4. Compute the RMSD. 5. Compute the radius of gyration over time. 6. Compute the SASA (solvent accessible surface area).',
 'Simulate 1LYZ for 1ps at 300 K. Report the secondary structure assignments of the downloaded PDB structure, and compute the RMSD, SASA, and radius of gyration of the simulation.')

In [3]:
llm_model = "claude-3-opus-20240229"
tools = "all"

In [4]:
agent_1 = MDAgent(agent_type="Structured", model=llm_model, top_k_tools=tools, use_memory=False)
agent_1.run(prompt_6_descriptive)

Thought: To complete the tasks, I will need to download the PDB file, analyze its structure, run a short simulation, and compute various properties. I will start by downloading the PDB file.

Action:
```
{
  "action": "PDBFileDownloader",
  "action_input": "1LYZ"
}
```

PDB file found with this ID: 1LYZ
Thought: Now that I have downloaded the PDB file, I can analyze the secondary structure using the ComputeDSSP tool.

Action:
```
{
  "action": "ComputeDSSP",
  "action_input": {
    "traj_file": "1LYZ_004453",
    "top_file": "1LYZ_004453",
    "target_frames": "first"
  }
}
```

Thought: Now that I have the secondary structure information, I can set up and run a short 1ps simulation at 300K using the SetUpandRunFunction tool.

Action:
```
{
  "action": "SetUpandRunFunction",
  "action_input": {
    "pdb_id": "1LYZ_004453",
    "forcefield_files": ["amber14/protein.ff14SB.xml", "amber14/tip3p.xml"],
    "save": true,
    "system_params": {
      "nonbondedMethod": "NoCutoff",
      "con

TypeError: ComputeRMSD._run() missing 1 required positional argument: 'top_id'

In [10]:
registry_1 = agent_1.path_registry
print(registry_1.list_path_names_and_descriptions().replace(",", "\n"))

Files found in registry: 1LYZ_004453: PDB file downloaded from RSCB
 PDBFile ID: 1LYZ_004453
 rec0_004500: dssp values for trajectory with id: 1LYZ_004453
 1LYZ_004525: Cleaned File:  Removed Heterogens
 and Water Removed.  Replaced Nonstandard Residues. Added Hydrogens at pH 7.0. Missing Atoms Added and replaces nonstandard residues. 
 top_sim0_004537: Initial positions for simulation sim0_004537
 sim0_004537: Basic Simulation of Protein 1LYZ_004525
 rec0_004545: Simulation trajectory for protein 1LYZ_004525 and simulation sim0_004537
 rec1_004545: Simulation state log for protein 1LYZ_004525 and simulation sim0_004537
 rec2_004545: Simulation pdb frames for protein 1LYZ_004525 and simulation sim0_004537
 rec0_005837: dssp values for trajectory with id: 1LYZ_004453


In [6]:
# make sure pdb was downloaded
assert os.path.exists(registry_1.get_mapped_path("1LYZ_004453"))

In [7]:
# make sure dssp was computed correctly
from mdagent.tools.base_tools import ComputeDSSP

dssp = ComputeDSSP(registry_1)
dssp._run(traj_file= "1LYZ_004453", target_frames="first")

"{'residues in helix': 50, 'residues in strand': 14, 'residues in coil': 65, 'residues not assigned, not a protein residue': 101}"

In [8]:
# make sure trajectory and topology exist
traj_path_1 = registry_1.get_mapped_path("sim0_004537")
top_path_1 = registry_1.get_mapped_path("top_sim0_004537")

assert os.path.exists(traj_path_1)
assert os.path.exists(top_path_1)

In [11]:
agent_2 = MDAgent(agent_type="Structured", model=llm_model, top_k_tools=tools, use_memory=False)
agent_2.run(prompt_6_natural)

Thought: To simulate 1LYZ, I first need to download the PDB file. I can use the PDBFileDownloader tool to get the 1LYZ PDB file.

Action:
```
{
  "action": "PDBFileDownloader",
  "action_input": "1LYZ"
}
```

PDB file found with this ID: 1LYZ
Thought: Now that I have the 1LYZ PDB file, I can compute the secondary structure assignments using the ComputeDSSP tool before running the simulation.

Action:
```
{
  "action": "ComputeDSSP",
  "action_input": {
    "top_file": "1LYZ_005953",
    "target_frames": "first"
  }
}
```



TypeError: ComputeDSSP._run() missing 1 required positional argument: 'traj_file'

In [12]:
registry_2 = agent_2.path_registry
print(registry_2.list_path_names_and_descriptions().replace(",", "\n"))

Files found in registry: 1LYZ_005953: PDB file downloaded from RSCB
 PDBFile ID: 1LYZ_005953


In [13]:
# make sure pdb was downloaded
assert os.path.exists(registry_2.get_mapped_path("1LYZ_005953")) 